## 1. Environment Setup (Linux)

### 1.1. Update Linux Packages

In [1]:
!sudo apt update

Hit:1 https://dl.yarnpkg.com/debian stable InRelease
0% [Waiting for headers] [Waiting for headers] [Connecting to archive.ubuntu.co

Hit:2 https://repo.anaconda.com/pkgs/misc/debrepo/conda stable InRelease       
Hit:3 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble InRelease
Hit:4 http://archive.ubuntu.com/ubuntu noble InRelease                       
Hit:5 http://archive.ubuntu.com/ubuntu noble-updates InRelease               
Hit:6 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
30 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [15]:
!sudo apt upgrade -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Calculating upgrade... Done
The following packages will be upgraded:
  binutils binutils-common binutils-x86-64-linux-gnu gir1.2-glib-2.0
  libapparmor1 libbinutils libctf-nobfd0 libctf0 libgirepository-2.0-0
  libglib2.0-0t64 libglib2.0-bin libglib2.0-data libglib2.0-dev
  libglib2.0-dev-bin libgprofng0 libmysqlclient-dev libmysqlclient21
  libpng-dev libpng16-16t64 libpq-dev libpq5 libsframe1 libunbound8
  linux-libc-dev moby-cli moby-compose moby-engine moby-runc python-apt-common
  python3-apt
30 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Need to get 63.0 MB of archives.
After this operation, 61.6 MB disk space will be freed.
Get:1 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble/main amd64 moby-runc amd64 1.4.0-ubuntu24.04u1 [3565 kB]
Get:2 https://packages.microsoft.com/repos/microsoft-ubuntu-noble-prod noble/main amd64 moby-engine amd64 28.5.2-

### 1.2. Install Python and Packages (pip & venv)

In [16]:
!sudo apt install -y python3 python3-pip python3-venv

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3 is already the newest version (3.12.3-0ubuntu2.1).
python3-pip is already the newest version (24.0+dfsg-1ubuntu1.3).
python3-venv is already the newest version (3.12.3-0ubuntu2.1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


### 1.3. Create a Virtual Environment

In [17]:
!python3 -m venv venv

### 1.4. Activate the Virtual Environment

In [18]:
!source venv/bin/activate

### 1.5. Install the Required Python Packages

In [19]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## 2. Data Ingestion

### 2.1. Download the Dataset

In [ ]:
!wget --user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64)" \
"https://business.yelp.com/external-assets/files/Yelp-JSON.zip" \
-O Yelp-JSON.zip


### 2.2. Unzip the downloaded file

In [ ]:
# Unzip the downloaded file
!unzip -o Yelp-JSON.zip

In [ ]:
# Create Datasets directory
!mkdir Datasets

In [ ]:
# Move the extracted tar file to Datasets directory
!tar -xvf Yelp\ JSON/yelp_dataset.tar -C Datasets

In [ ]:
# Remove unnecessary files
!rm Yelp-JSON.zip
!rm -rf __MACOSX/
!rm -rf Yelp\ JSON/

## 3. Data Cleaning

In [20]:
import pyspark
from pathlib import Path
from pyspark.sql import SparkSession

# In notebooks, re-running cells can try to create a second SparkContext.
# Reuse the active SparkSession if it already exists.
spark = SparkSession.getActiveSession()
if spark is None:
    spark = (
        SparkSession.builder
        .appName("Yelp Data Analysis")
        .master("local[*]")
        .getOrCreate()
    )

print("Spark version:", spark.version)

25/12/12 18:51:36 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:483)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.ja

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is not supported
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:277)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:588)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2446)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2446)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:339)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:483)
	at py4j.reflection.MethodInvoker.in

## 4. Data Transformation

## 5. Data Querying